In [1]:


import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.cluster import KMeans
from scipy.stats import kendalltau
import os.path
import winsound
from imblearn.over_sampling import SMOTE
from scipy.stats import loguniform
import sys
from sklearn.model_selection import StratifiedShuffleSplit
import os
from scipy.stats import chi2_contingency
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, f1_score
import os.path
from openpyxl import Workbook, load_workbook
import xgboost as xgb
from scipy.stats import randint
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from scipy.stats import pearsonr
from sklearn.svm import SVC
from pandas_profiling import ProfileReport
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from scipy.stats import pointbiserialr
from tabulate import tabulate
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.cluster import KMeans
import datetime
import sys
import os
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from pandas_profiling import ProfileReport
from scipy.stats import f_oneway, shapiro
from io import BytesIO
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
import seaborn as sns
import dtale
from scipy.stats import chi2_contingency
from scipy import stats
import matplotlib.cm
import re
import base64
from scipy.stats import pointbiserialr, pearsonr
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
import category_encoders as ce
from lightgbm import LGBMRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import winsound

plt.rcParams['figure.facecolor'] = 'white'
np.set_printoptions(threshold=sys.maxsize, suppress=True)

In [2]:
datatypes = {'business_area': str, 'sex': str, 'employment_status': str,
            'time': str, 'time_ESAW': str, 'severity': str, 'enterprise_size': str,
            'citizenship': str, 'profession_code': str, 'type_of_injury': str
            , 'injured_bodypart': str, 'workstation': str,
            'working_environment': str, 'working_process': str
            , 'specific_physical_activity': str,
            'material_agent_of_physical_act.': str,
            'deviation': str, 'material_agent_of_deviation': str
            , 'contact_mode_of_injury': str, 'material_agent_of_contact_mode': str, 'general_profession_code': str, 'month': str, 'weekofyear': str, 'dayofweek': str, 'material_agent_of_physical_act': str}

In [3]:
# For reproducing results use data.csv in your working directory and comment the fist line out
os.chdir('C:/Users/Mario/OneDrive - Tartu Ülikool/IT_mitteinformaatikutele_MSc/Kevad 2023/Student_Project_Contest_UT_2023/Code')
df = pd.read_csv("data.csv", encoding='latin-1', dtype = datatypes, index_col=0)
df = df.drop(columns= ['date', 'time', 'time_ESAW', 'datetime', 'lost_days', 'type_of_injury', 'injured_bodypart', 'severity'])

In [4]:
sectors = list(df.business_sector.unique())
classes = list(df.general_profession_class.unique())

In [5]:
dff = df.copy()
for f in sectors:

    for e in classes:
        
        df = dff.copy()
        df = df[(df['business_sector'] == f)]
        df = df[(df['general_profession_class'] == e)]
        if (df['target'] == 1).sum() < 5:
            print('Not enough positive samples for sector: ' + f + ' and class: ' + str(e))
            continue
        df_sub = df.drop(columns=['business_sector', 'general_profession_class'])

        df_sub_copy = df_sub.copy()

        # --------------------------------------------------------------------------------
        #TRAINING ON SIGNIFICANT FEATURES
        # --------------------------------------------------------------------------------

        def encode_categorical(df, categorical_columns):
            le = LabelEncoder()
            for col in categorical_columns:
                df[col] = le.fit_transform(df[col])
            return df

        categorical_columns = df_sub.select_dtypes(include='object').columns.tolist()
        df_encoded = encode_categorical(df_sub, categorical_columns)

        # Get the column names with 'int32' and 'int64' dtypes
        int32_columns = df_encoded.select_dtypes(include='int32').columns.tolist()
        int64_columns = df_encoded.select_dtypes(include='int64').columns.tolist()

        # Combine the two lists to create the categorical_columns list
        categorical_columns = int32_columns + int64_columns

        def chi_squared_test(df, target, categorical_columns):
            significant_features = []
            for col in categorical_columns:
                contingency_table = pd.crosstab(df[col], df[target])
                chi2, p_value, _, _ = stats.chi2_contingency(contingency_table)
                
                if p_value < 0.05:
                    significant_features.append(col)
            
            return significant_features

        significant_chi2_features = chi_squared_test(df_encoded, 'target', categorical_columns)


        def kendall_correlation(df, target, numerical_columns):
            significant_features = []
            for col in numerical_columns:
                kendall_tau, p_value = stats.kendalltau(df[col], df[target])
                
                if p_value < 0.05:
                    significant_features.append(col)
            
            return significant_features

        numerical_columns = df_sub.select_dtypes(include='float64').columns.tolist()
        significant_kendall_features = kendall_correlation(df_sub, 'target', numerical_columns)

        significant_features = significant_chi2_features + significant_kendall_features

        df = df_sub_copy[significant_features]

        # Encode categorical features with no order
        all_object_cols = list(df.select_dtypes(include=['object']).columns)
        df = pd.get_dummies(df, columns=all_object_cols)

        # Separate independent variables and dependent variable for train set
        X = df.drop(columns=['target'])
        y = df['target']

        # Balance the target class using random over-sampling
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        try:
            float_cols = X_train.select_dtypes(include='float64').columns.tolist()
            scaler = MinMaxScaler(feature_range=(0,1))
            X_train[float_cols] = scaler.fit_transform(X_train[float_cols])
            X_test[float_cols] = scaler.transform(X_test[float_cols])
        except ValueError as e:
            print(f"An error occurred: {e}")
            continue

        feat = 'significant_features'

        # ----------------------------------------------------------------------------
        #TRAINING ON ALL FEATURES
        # ----------------------------------------------------------------------------


        # Encode categorical features with no order
        """ all_object_cols = list(df.select_dtypes(include=['object']).columns)
        df = pd.get_dummies(df, columns=all_object_cols)

        # Separate independent variables and dependent variable for train set
        X = df.drop(columns=['target'])
        y = df['target']

        # Balance the target class using random over-sampling
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Scale float64 variables
        float_cols = X_train.select_dtypes(include='float64').columns.tolist()
        scaler = MinMaxScaler(feature_range=(0,1))
        X_train[float_cols] = scaler.fit_transform(X_train[float_cols])
        X_test[float_cols] = scaler.transform(X_test[float_cols])


        feat = 'all_features' """

        # ---------------------------------------------------------------------------------------------------------------------------


        nr_of_independent_var = len(X_train.columns)
        independent_var_names = [col for col in X_train.columns if col != 'target']
        target_0 = y_train.value_counts()[0]
        target_1 = y_train.value_counts()[1]

        # Calculate the proportion of positive target class in the y_test data
        positive_class_count = y_test.sum()
        total_count = len(y_test)
        proportion = positive_class_count / total_count

        # Calculate the 95% confidence interval for the proportion
        z = stats.norm.ppf(0.975)  # 95% confidence level
        margin_of_error = z * np.sqrt((proportion * (1 - proportion)) / total_count)
        confidence_interval = (proportion - margin_of_error, proportion + margin_of_error)

        # Define a threshold for an acceptable margin of error (you can adjust this value)
        acceptable_margin_of_error = 0.1

        # Print evaluation sentence
        if margin_of_error <= acceptable_margin_of_error:
            print(f"Margin of error: PASS for {f}{e}")
        else:
            print(f"Margin of error: FAIL for {f}{e}")
            continue
    

        # Use selected resampling technique
        
        """ over_sampler = RandomOverSampler(random_state=42)
        X_train, y_train = over_sampler.fit_resample(X_train, y_train)
        balancing_tech = 'ROS' """
        
        """ under_sampler = RandomUnderSampler(random_state=42)
        X_train, y_train = under_sampler.fit_resample(X_train, y_train)
        balancing_tech = 'RUS' """

        try:
            smote = SMOTE(random_state=rs)
            X_train, y_train = smote.fit_resample(X_train, y_train)
            balancing_tech = 'SMOTE'
        except ValueError as e:
            print(f"An error occurred in SMOTE balancing: {e}")
            continue


          # Check if the Excel file already exists
        file_exists = os.path.isfile(f'combinations_{feat}_{balancing_tech}.xlsx')

        # If the file doesn't exist, create a new DataFrame to store the results
        if not file_exists:
            results_df = pd.DataFrame(columns=[
                "Sector",
                "Class",
                "Nr_of_ind._var",
                "Names_of_ind._var",
                "Target_0_count",
                "Target_1_count",
                "Model_name",
                "Balancing_tech.",
                "AUCROC",
                "CV_AUCROC",
                "CV_AUCROC_std",
                "AVG_F1",
                "F1_score_0",
                "F1_score_1",
                "Precision_0",
                "Precision_1",
                "Recall_0",
                "Recall_1",
            ])
        else:
            # If the file exists, load the existing DataFrame from the file
            results_df = pd.read_excel(f'combinations_{feat}_{balancing_tech}.xlsx')

        rs = 42
        rf_model = RandomForestClassifier(random_state=rs)
        lgbm_model = LGBMClassifier(random_state=rs)
        svm_model = SVC(kernel='rbf', probability=True, random_state=rs)
        xgb_model = xgb.XGBClassifier(random_state=rs)
        lr_model = LogisticRegression(max_iter=1000, random_state=rs)

        models = [rf_model, lgbm_model, xgb_model, svm_model, lr_model]
        model_names = ['Random Forest', 'LightGBM', 'XGBoost', 'SVM', 'Logistic Regression']

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rs)   

        for k, model in enumerate(models):
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred_prob = model.predict_proba(X_test)[:,1]
            
            try:
                auc_roc = roc_auc_score(y_test, y_pred_prob)
            except ValueError:
                print(f"Skipping {model_names[k]} for {f}{e} due to only one class present in y_test")
                continue
            acc = accuracy_score(y_test, y_pred)
            report = classification_report(y_test, y_pred)
            auc_roc = roc_auc_score(y_test, y_pred_prob)
            fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
            f1_class_0 = f1_score(y_test, y_pred, pos_label=0)
            f1_class_1 = f1_score(y_test, y_pred, pos_label=1)
            avg_f1 = f1_score(y_test, y_pred, average='weighted')
            
            precision_class_0 = precision_score(y_test, y_pred, pos_label=0)
            precision_class_1 = precision_score(y_test, y_pred, pos_label=1)
            recall_class_0 = recall_score(y_test, y_pred, pos_label=0)
            recall_class_1 = recall_score(y_test, y_pred, pos_label=1)

            try:
                balancing_tech = balancing_tech
            except NameError:
                balancing_tech = None
            try:
                comment1 = comment1
            except NameError:
                comment1 = None
            # Create a dictionary of results
            result = {
                "Sector": f,
                "Class": e,
                "Nr_of_ind._var": nr_of_independent_var,
                "Names_of_ind._var": ", ".join(independent_var_names),
                "Target_0_count": target_0,
                "Target_1_count": target_1,
                "Model_name": model_names[k],
                "Balancing_tech.": balancing_tech,
                "AUCROC": auc_roc,
                "CV_AUCROC": np.mean(cv_scores),
                "CV_AUCROC_std": np.std(cv_scores),
                "AVG_F1": avg_f1,
                "F1_score_0": f1_class_0,
                "F1_score_1": f1_class_1,
                "Precision_0": precision_class_0,
                "Precision_1": precision_class_1,
                "Recall_0": recall_class_0,
                "Recall_1": recall_class_1
            }

            # Append the result to the results DataFrame
            results_df = pd.concat([results_df, pd.DataFrame(result, index=[0])], ignore_index=True)
            print(f"Salvestasin excelisse: {f}{e}")
            # Save the results DataFrame to the Excel file
            results_df.to_excel(f'combinations_{feat}_{balancing_tech}.xlsx', index=False)

Margin of error: PASS for C9


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\Users\Mario\AppData\Local\Programs\PythonCodingPack\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-5e2506b37d41>", line 154, in <module>
    smote = SMOTE(random_state=rs)
NameError: name 'rs' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Mario\AppData\Local\Programs\PythonCodingPack\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Mario\AppData\Local\Programs\PythonCodingPack\lib\site-packages\IPython\core\ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, num

TypeError: object of type 'NoneType' has no len()